## Start the NLP server


In [2]:
import os

STANFORD_NLP_LOCATION = '~/stanford-corenlp-full-2018-10-05/' # the location of the Stanford NLP library on my computer 
                                                              #(download at https://stanfordnlp.github.io/CoreNLP)
STANFORD_NLP_TIMEOUT = 10000 # the time after which a given NLP request will be killed if not yet complete

exec = os.popen
exec('cd %s; java -mx5g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout %s' % 
     (STANFORD_NLP_LOCATION, STANFORD_NLP_TIMEOUT))

## Create function to determine the sentiment of a block of text

In [75]:
from pycorenlp import StanfordCoreNLP
import numpy

MIN_SENTIMENT = 0
MAX_SENTIMENT = 4

# Determines the sentiment of a given block of text as a range from 0 (Extremely Negative) to 4 (Extremely Positive).
def findSentiment(text):
    NLP_SERVER_LOCATION = 'http://localhost:9000'
    PROPERTIES_DICTIONARY = {'annotators': 'sentiment', 'outputFormat': 'json', 'timeout': 1000}
    
    nlp = StanfordCoreNLP(NLP_SERVER_LOCATION)
    result = nlp.annotate(text, properties = PROPERTIES_DICTIONARY)

    sentiments = []
    for sentenceAnalysis in result['sentences']:
        sentence = " ".join(t['word'] for t in sentenceAnalysis['tokens'])
        sentence = sentence.replace('.', '')

        sentimentValue = float(sentenceAnalysis['sentimentValue'])

        sentiments += [sentimentValue]
    return numpy.average(sentiments)

print(findSentiment(TEXT))

1.3571428571428572


## Create gauge visual for sentiment reading

In [121]:
import plotly.offline as py
import plotly.graph_objs as go
import math

def map(num, init_min, init_max, new_min, new_max):
    return (num - init_min) * (new_max - new_min) / (init_max - init_min) + new_min

py.init_notebook_mode(connected=True) # Include this line for all charts to be displayed directly in the notebook

base_chart = {
    "values": [50, 10, 10, 10, 10, 10],
    "labels": ["-", "-", "-", "-", "-", "-"],
    "domain": {"x": [0, .48]},
    "marker": {
        "colors": [
            'rgb(255, 255, 255)',
            'rgb(255, 255, 255)',
            'rgb(255, 255, 255)',
            'rgb(255, 255, 255)',
            'rgb(255, 255, 255)',
            'rgb(255, 255, 255)',
            'rgb(255, 255, 255)'
        ],
        "line": {
            "width": 0
        }
    },
    "name": "Sentiment Gauge",
    "hole": 0.4,
    "type": "pie",
    "direction": "clockwise",
    "rotation": 108,
    "showlegend": False,
    "hoverinfo": "none",
    "textinfo": "label",
    "textposition": "outside"
}

meter_chart = {
    "values": [50, 10, 10, 10, 10, 10],
    "labels": ["-", "Extremely Negative", "Negative", "Neutral", "Positive", "Extremely Positive"],
    "marker": {
        'colors': [
            'rgb(255, 255, 255)',
            'rgb(200,0,0)',
            'rgb(200,75,0)',
            'rgb(100,100,0)',
            'rgb(100,160,0)',
            'rgb(0,200,0)'
        ]
    },
    "name": "Gauge",
    "hole": .3,
    "type": "pie",
    "direction": "clockwise",
    "rotation": 90,
    "showlegend": False,
    "textinfo": "label",
    "textposition": "inside",
    "hoverinfo": "none"
}

CIRCLE_RADIUS = 0.15
CIRCLE_CENTER = (0.5, 0.5)
TRIANGLE_SIDE_LENGTH = 0.02

circleEqn = lambda x : (math.sqrt(abs(math.pow(CIRCLE_RADIUS, 2) - math.pow((x - CIRCLE_CENTER[0]), 2))) + CIRCLE_CENTER[1])

xVal = map(3, MIN_SENTIMENT, MAX_SENTIMENT, CIRCLE_CENTER[0] - CIRCLE_RADIUS, CIRCLE_CENTER[0] + CIRCLE_RADIUS)
yVal = circleEqn(xVal)
properties = {
    "xaxis": {
        "showticklabels": False,
        'showgrid': False,
        'zeroline': False,
    },
    "yaxis": {
        "showticklabels": False,
        'showgrid': False,
        'zeroline': False,
    },
    "shapes": [
        {
            "type": "path",
            "path": "M 0.495 0.5 L %s %s L 0.505 0.5 Z" % (xVal, yVal), # specify path in SVG format
            "fillcolor": "rgba(44, 160, 101, 0.5)",
            "line": {
                "width": 0.5
            }
        }
    ],
}

gauge = {
    "data": [base_chart, meter_chart],
    "layout": properties
}

py.iplot(gauge)

## Scrape web for news article

In [10]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from newsapi import NewsApiClient

def getPageContent(url):
    try: 
        with closing(get(url, stream = True)) as page:
            return page.content.decode("utf-8")
    except RequestException as e:
        print(e)
        return
    
def parseHTML(url):
    return BeautifulSoup(getPageContent(url), 'html.parser')

# Extracts the text from a CNN article with given URL, excluding the headline and any advertisements.
def getCNNText(url):
    htmlParser = parseHTML(url)
    
    text = ""

    for element in htmlParser.select('div'):
        if element.has_attr('class') and 'zn-body__paragraph' in element['class']: 
            text += element.text 
    return text
text = getCNNText("https://www.cnn.com/2019/06/26/business/bitcoin-value-trnd/index.html")
print(text)
print(findSentiment(text))

# Uses NewsAPI to extract all articles from a given day.
def getArticleURLs(day, month, year):
    dayPrefix = "%04d-%02d-%02d" % (year, month, day)
    
    newsapi = NewsApiClient(api_key = '8d99d69a251a453f8c084f4768db7195')
    
    urls = []
    second = 0
    minute = 0
    for hour in range (0, 24):
        for minute in range (0, 59, 10):
            iso8601DateStart = "%sT%s" % (dayPrefix, "%02d:%02d:%02d" % (hour, minute, second))
            iso8601DateEnd = "%sT%s" % (dayPrefix, "%02d:%02d:%02d" % (hour, minute + (10 if minute != 50 else 9), second))
            all_articles = newsapi.get_everything(q='bitcoin', from_param= iso8601DateStart, to=iso8601DateEnd, page=1)
            for article in all_articles['articles']:
                urls += article['url']
    return urls

Its value surged on Wednesday, passing the $13,000 mark, before it shed about $1,000 within minutes. Around 6 pm ET, the currency was back to a $12,400-plus valuation.The reason for the sudden midday sell off wasn't immediately clear. But cryptocurrencies are known for price swings. A previous bitcoin craze hit a fever pitch in 2017. Bitcoin soared from just a few hundred dollars apiece in January 2017 to more than $20,000. But it came crashing down again. BItcoin's value was still hovering around the $3,000 mark four months ago.  Experts say a weakening US dollar and fresh discussions about regulating digital currencies are fueling the latest spike in demand.BItcoin reached a $10,000 value on Friday for the first time in more than a year. It was likely helped by an announcement from Facebook (FB) last month that the social network planned to debut its own digital currency, called Libra, introducing its 2 billion users to crypto. Libra may hurt bitcoin's value if it becomes the preferr

## Kill the NLP server

In [247]:
exec('kill $(lsof -ti tcp:9000)')